**IMPORT LIBRARY**

In [ ]:
import pandas as pd
import re
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
from sklearn.model_selection import train_test_split
from sentence_transformers import SentenceTransformer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.exceptions import ConvergenceWarning

**DOWNLOAD DATASET**

In [ ]:
from google.colab import files
files.upload()

"""
Buat folder baru bernama kaggle untuk menyimpan API Token dari Kaggle.com
"""
!rm -r ~/.kaggle
!mkdir ~/.kaggle
!mv ./kaggle.json ~/.kaggle/

# Change permission of the site
!chmod 600 ~/.kaggle/kaggle.json

# Download dataset
!kaggle datasets download -d "farhan999/tokopedia-product-reviews"

# Unzip file
!unzip -o tokopedia-product-reviews.zip

**LOAD DATASET**

In [ ]:
tokped_reviews = pd.read_csv("/content/tokopedia-product-reviews-2019.csv")
tokped_reviews

**NILAI KONDISI DATA**

In [ ]:
tokped_reviews.info()

In [ ]:
tokped_reviews.isna().sum()

In [ ]:
tokped_reviews.duplicated().sum()

**INSIGHT**

- Ditemukan missing value pada kolom sold sebanyak 14 data

- Dari kolom yang tersedia, akan diambil 2 kolom saja yaitu kolom **text dan rating** untuk dijadikan kolom baru bernama label

pada kolom text ditemukan:

*   **campuran penulisan antara huruf besar huruf kecil**
*   **komen yang menggunakan emoji**
*   **komen yang menggunakan karakter atau tanda baca > 1 secara berurutan**
*   Penggunaan 2 untuk mewakili **perulangan kalimat** seperti hati2
*   Kolem komen yang hanya berisikan **simbol atau tanda baca saja**

**PREPROCESSING DATA**

In [ ]:
def remove_emoji(text):
  """
  Fungsi ini digunakan untuk menghapus emoji dam simbol kode Unicode yang ada di dalam text

  Args:
    text: Input dari kolom teks yang akan dibersikan

  Returns:
    str: String yang sudah bersih dari emoji dan simbol kode Unicode
  """
  emoji_pattern = re.compile("["
                            "\U0001F600-\U0001F64F" # Emoji
                            "\U0001F300-\U0001F5FF" # Simbol
                             "]+",
                             flags=re.UNICODE
                            )
  return emoji_pattern.sub(r'', str(text))

In [ ]:
def load_slang_dict():
    """
    Fungsi ini mengembalikan kamus yang berisikan kata slang dan kata baku.

    Kamus ini mencakup:
    - Singkatan
    - Typo
    - Simbol
     - Karakter emoticon

    Args:
        None

    Returns:
        dict: Kamus slang custom yang berisi kata slang dan kata baku.
    """
    # Kamus slang custom
    kamus_baku = {
        # Kalimat
        "dgn": "dengan", "blum": "belum", "blm": "belum", "ndak": "tidak", "tsk": "tidak",
        "tdk": "tidak", "wrn": "warna", "krg": "kurang", "brg": "barang", "mantab": "mantap",
        "sy": "saya", "dscrbs": "deskripsi", "describe":" deskripsi", "hny": "hanya","sdh": "sudah",
        "dr": "dari", "bgmn":"bagaimana", "dech": " ", "deh": " ", "tp": "tapi","tpi": "tapi",
        "yg": "yang", "&": "dan", "\n": " ", "god.": "bagus", "kl": "kalau", "klo": "kalau",
        "smga": "semoga", "smoga": "semoga", "trus": "terus", "danamp;": "dan", "dos": "kotak",
        "dus": "kotak", "prosen": "proses", "thankss": "terima kasih", "thanks": "terima kasih",
        "thx": "terima kasih","seler": "penjual", "seller": "penjual", "paking": "kemasan",
        "peking": "kemasan","langanan": "langganan", "buat": "untuk", "pokok e": "pokoknya",
        "jos": "mantap", "pengriman": "pengiriman", "sih": " ", "gimana": "bagaimana",
        "gmn": "bagaimana", "apaan": "apa", "jlk": "jelek", "ih": " ", "nih": " ", "jd": "jadi",
        "gk": "tidak", "bgt": "sangat", "banget": "sangat", "dong": " ", "lah": " ",

        # Emoji
        ":)": " senang ", ":(": " sedih "
    }

    return kamus_baku

In [ ]:
def remove_repeated_char(text):
  """
  Fungsi ini menggunakan regex untuk menemukan urutan karakter yang muncul lebih dari 1x
  secara berurutan. Pola (.) akan menangkap karakter dan \1+ akan mencocokan apakah karakter
  tersebut diulang satu kali atau lebih

  Args:
    text (string): Input teks yang akan dibersihkan

  Return:
    str:
  """
  pattern = re.compile(r'(.)\1+')
  return pattern.sub(r'\1', str(text))

In [ ]:
def normalisasi_duplikasi(text):
  """
  Fungsi ini digunakan untuk merubah angka 2 pada kalimat (misal: "hati2") menjadi
  bentuk formal (misal: "hati-hati")

  Args:
    text (string): Input teks yang akan dibersihkan

  Return:
    str: Teks yang sudah dibersihkan
  """
  pattern = re.compile(r'([a-zA-Z]+)2')
  return pattern.sub(r'\1-\1', str(text))

In [ ]:
def remove_punctuation(text):
    """
    Menghapus SEMUA tanda baca KECUALI hyphen (-) internal
    dan menggantinya dengan spasi.
    """
    pattern = re.compile(r'[^\w\s-]')
    return pattern.sub(' ', str(text))

In [ ]:
def labeling(rating):
  """
  Fungsi ini mengambil nilai integer dari kolom rating dan mengubahnya menjadi tiga
  label string:
    - positif
    - netral
    - negatif

  Args:
    rating (int): Nilai integer dari kolom rating yang akan diubah menjadi label

  Returns:
    str: Label sentiment ('positif', 'netral', 'negatif') berdasarkan nilai rating
  """
  if rating >= 4:
    return "positif"
  elif rating == 3:
    return "netral"
  elif rating <=2:
    return "negatif"

In [ ]:
# Hapus kolom yang tidak digunakan
tokped_cleaned = tokped_reviews.drop(columns=['Unnamed: 0', 'category', 'product_name', 'product_id', 'sold', 'shop_id', 'product_url'])

In [ ]:
# Ubah menjadi huruf kecil untuk semua isi kolom text
tokped_cleaned['text'] = tokped_cleaned['text'].str.lower()

# Hapus baris yang tidak berisikan huruf sama sekali
mask = tokped_cleaned['text'].apply(lambda x: bool(re.search('[a-zA-Z]', str(x))))
tokped_cleaned = tokped_cleaned[mask]

In [ ]:
# Menerapkan remove emoji
tokped_cleaned['text'] = tokped_cleaned['text'].apply(remove_emoji)

In [ ]:
# Load kamus
kamus_baku = load_slang_dict()

# emoji :) dan :( terbaca sebagai string bukan perintah
kamus_baku = {re.escape(k): v for k, v in kamus_baku.items()}

# Menerapkan normalisasi kata slang
tokped_cleaned['text'] = tokped_cleaned['text'].replace(kamus_baku, regex=True)

In [ ]:
tokped_cleaned['text'] = tokped_cleaned['text'].apply(remove_punctuation) # Hapus tanda baca

In [ ]:
# Menghapus karakter yang muncul > 1 secara berurutan
tokped_cleaned['text'] = tokped_cleaned['text'].apply(remove_repeated_char)

# Normalisasi angka 2 pada kalimat menjadi formal
tokped_cleaned['text'] = tokped_cleaned['text'].apply(normalisasi_duplikasi)

In [ ]:
# Buat kolom baru bernama label
tokped_cleaned['label'] = tokped_cleaned['rating'].apply(labeling)

# Hapus kolom rating karena sudah tidak digunakan lagi
tokped_cleaned = tokped_cleaned.drop(columns=['rating'])

In [ ]:
tokped_cleaned.head(30)

In [ ]:
# Simpan dataset yang sudah bersihkan ke format csv
tokped_cleaned.to_csv("tokopedia_reviews_clean.csv", index=False)

In [ ]:
tokped_cleaned = pd.read_csv("/content/tokopedia_reviews_clean.csv")

In [ ]:
# Label Encoding kolom label
label_mapping = {
    "positif": 2,
    "netral": 1,
    "negatif": 0
}

# Terapkan Encodin
tokped_cleaned['label'] = tokped_cleaned['label'].map(label_mapping)

**Splitting Dataset**

In [ ]:
tokped_cleaned.head()

In [ ]:
# Memisahkan antara kolom target (y) dan kolom input (X)
X = tokped_cleaned['text']
y = tokped_cleaned['label']

# splitting 80% data latih dan 20% data test
X_train, X_test, y_train, y_test = train_test_split(X, y,
                                                    test_size=0.2,
                                                    random_state=42,
                                                    stratify=y)

**SENTENCE-TRANSFOMER**

In [ ]:
# Load model
model = SentenceTransformer('paraphrase-multilingual-MiniLM-L12-v2')

# Ubah data latih di X_train kedalam bentuk vektor
X_train_embedding = model.encode(X_train.tolist(), show_progress_bar=True)

# Ubah data uji di X_test kedalam bentuk vektor
X_test_embedding = model.encode(X_test.tolist(), show_progress_bar=True)

print(f"Data Latih: {X_train_embedding.shape}")
print(f"Data Uji: {X_test_embedding.shape}")

**TRAIN MODEL**

In [ ]:
# Abaikan jumlah max_iter jika mendekati titik optimal
warnings.filterwarnings("ignore", category=ConvergenceWarning)

# Load model logisticRegression
lr_classifier = LogisticRegression(max_iter=100, random_state=42)

# Train model
lr_classifier.fit(X_train_embedding, y_train)

# Hasil Akurasi data latih
y_train_pred = lr_classifier.predict(X_train_embedding)
train_accuracy = accuracy_score(y_train, y_train_pred)
print(f"Akurasi Data Latih: {train_accuracy:.4f}")

# Hasil evaluasi
print(classification_report(y_train, y_train_pred, target_names=["negatif (0)",
                                                          "netral (1)",
                                                          "positif (2)"]
                            ))

In [ ]:
# Inisialisasi confusion matrix
cm = confusion_matrix(y_train, y_train_pred)

# Label
labels = ["negatif (0)", "netral (1)", "positif (2)"]

# plot confusion matrix
plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True,
            fmt='d',
            cmap='Blues',
            xticklabels=labels,
            yticklabels=labels)

plt.title("Confusion Matrix - Data Latih")
plt.xlabel("Label Prediksi")
plt.ylabel("Label Aktual")
plt.show()

**TEST MODEL**

In [ ]:
# Prediksi model pada data uji
y_pred = lr_classifier.predict(X_test_embedding)

# Hasil Akurasi data uji
test_accuracy = accuracy_score(y_test, y_pred)
print(f"Akurasi Data Uji: {test_accuracy:.4f}")

# Hasil evaluasi
print(classification_report(y_test, y_pred, target_names=["negatif (0)",
                                                          "netral (1)",
                                                          "positif (2)"]
                            ))

In [ ]:
# Inisialisasi confusion matrix
cm = confusion_matrix(y_test, y_pred)

# Label
labels = ["negatif (0)", "netral (1)", "positif (2)"]

# plot confusion matrix
plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True,
            fmt='d',
            cmap='Blues',
            xticklabels=labels,
            yticklabels=labels)

plt.title("Confusion Matrix - Data Uji")
plt.xlabel("Label Prediksi")
plt.ylabel("Label Aktual")
plt.show()

**CONTOH OUTPUT**

In [ ]:
def preprocessing_text(text):
  """
  Alur preprocessing yang disesuaikan dengan preprocessing sebelumnya
  """
  text = str(text).lower() # Ubah menjadi lowercase
  text = remove_emoji(text) # Hapus emoji dari text
  text = remove_repeated_char(text) # Hapus karakter, tanda baca yg muncul > 1x berurutan
  text = normalisasi_duplikasi(text) # ubah kalimat hati2 -> hati-hati

  # Ubah kata slang menjadi bentuk baku
  temp_series = pd.Series([text])
  temp_series = temp_series.replace(kamus_baku, regex=True)
  text = temp_series.iloc[0]

  text = text.strip()
  text = re.sub(r'\s+', ' ', text)

  return text

In [ ]:
label_map = {
    2: "positif",
    1: "netral",
    0: "negatif"
}

def sentiment(text_input):
  print(f"Input: {text_input}")

  # Panggil fungsi preprocessing_text
  clean_text = preprocessing_text(text_input)

  # Buat embedding (Encoder)
  embedding_text = model.encode([clean_text])

  # Prediksi inputan text
  prediction = lr_classifier.predict(embedding_text)

  # Hasil prediksi
  label = prediction[0]

  # Ubah kedalam bentuk string
  label_string = label_map.get(label, "Unknown")

  print(f"Prediksi: {label_string} (Kode: {label})")

  return label, label_string

In [ ]:
sentiment("Barangnya bagus, cuman respon sellernya jelek banget")

**SAVE MODEL**

In [ ]:
import joblib

model_path = "model_classifier.joblib"

joblib.dump(lr_classifier, model_path)

In [ ]:
model_encoder_path = "sentence_transformer"

model.save(model_encoder_path)

In [ ]:
!zip -r sentence_transformer.zip sentence_transformer